In [1]:
import polars as pl
import numpy as np
import sqlite3
import json
from datetime import datetime
import matplotlib.pyplot as plt
import string
np.random.seed(152)

# Preprocessing

If necessary, adapt sqlite columns to VARCHAR by:
```
ALTER TABLE product ADD COLUMN new_price VARCHAR;
UPDATE product SET new_price = price;
ALTER TABLE product DROP COLUMN price;
ALTER TABLE product RENAME COLUMN new_price TO price;
```
```
ALTER TABLE review ADD COLUMN new_found_funny VARCHAR;
UPDATE review SET new_found_funny = found_funny;
ALTER TABLE review DROP COLUMN found_funny;
ALTER TABLE review RENAME COLUMN new_found_funny TO found_funny;
```

## Selecting Features

Selecting features is an art. Often more features translate into better predictions, but it also translates into more time and resource-intensive training and predicting. While you might use some statistical metrics like correlation to quickly see what features impact our predictions most, this gets harder when we have multiple text features. You of course could test how the addition of certain features impacts predictions, but that is again very time and resource-consuming process. It often comes down to which features you think might improve performance the most and then using those. 

In our case, we want to select only features that seem most relevant to our tasks. Since we are building a model that tries to predict review properties, the most important feature will be the review text. The second most important feature is probably the description of the product the review is related to. Other features seem to have less impact on our predictions. Nonetheless, we could add information about product title, publisher, developer, tags, specs, genre, and price. There might be other features that could contribute to better predictions, but we will stop ourselves here.

## Reading

In [2]:
# db_path = '../dbs/db_micro2.sqlite3'
db_path = '../dbs/db2.sqlite3'
connection_string = 'sqlite://' + db_path
df = pl.read_database_uri('SELECT product_id, text AS review_text, recommended, found_awarding, found_helpful, found_funny FROM review LEFT JOIN product ON product_id = product.id', connection_string)

In [5]:
df_tags = pl.read_database_uri('SELECT tag.name AS tag, product.id AS product_id FROM product_tag JOIN tag ON product_tag.tag_id = tag.id JOIN product ON product_tag.product_id = product.id', connection_string)
df_specs = pl.read_database_uri('SELECT spec.name AS spec, product.id AS product_id FROM product_spec JOIN spec ON product_spec.spec_id = spec.id JOIN product ON product_spec.product_id = product.id', connection_string)
df_genres = pl.read_database_uri('SELECT genre.name AS genre, product.id AS product_id FROM product_genre JOIN genre ON product_genre.genre_id = genre.id JOIN product ON product_genre.product_id = product.id', connection_string)

# add True values for pivot
df_tags = df_tags.with_columns(
  pl.lit(True).alias('val')
)
df_specs = df_specs.with_columns(
  pl.lit(True).alias('val')
)
df_genres = df_genres.with_columns(
  pl.lit(True).alias('val')
)

# create pivot tables
df_tags = df_tags.pivot('tag', index='product_id').fill_null(False)
df_specs = df_specs.pivot('spec', index='product_id').fill_null(False)
df_genres = df_genres.pivot('genre', index='product_id').fill_null(False)

In [6]:
df_genres

product_id,Casual,Indie,Strategy,Free to Play,Early Access,Action,Adventure,Simulation,Sports,RPG,Racing,Design &amp; Illustration,Web Publishing,Massively Multiplayer,Animation &amp; Modeling,Software Training,Education,Utilities,Game Development,Photo Editing,Audio Production,Video Production
i64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
2655790,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
2261640,true,true,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
1425860,true,true,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
1397940,false,true,false,false,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
1431620,true,true,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1412710,true,false,true,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false
1356410,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
381713,false,false,false,false,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false


# Data wrangling

### Investigate selected features

TODO:
- Add one-hot encodings to input data
- Build models
- Evaluate results

### Formatting data

#### Convert to integers

In [3]:
# convert to integers
df = df.with_columns(
    # cast features to minimal viable types
    pl.col("found_funny").cast(pl.UInt16, strict=False).fill_null(strategy="zero"),
    pl.col("found_awarding").cast(pl.UInt16, strict=False).fill_null(strategy="zero"),
    pl.col("found_helpful").cast(pl.UInt16, strict=False).fill_null(strategy="zero"),
    # pl.col("recommended").cast(pl.UInt8).cast(pl.Boolean),
    # pl.col("price").str.replace("€", "").cast(pl.Float32, strict=False).fill_null(strategy="zero")
)

#### Choosing and calculating regression metric

We are trying to predict how funny, helpful, and awarding a review is. We have insight into this because people on Steam vote for reviews in each category. The easiest metric we could use is to try to predict how many people found a review funny. The problem is, that some reviews have more views than others, so it is only logical that those viewed less have fewer votes. Unfortunately, we don't have information about the number of views of a review, so we have to find another way to consider this metric.

We could do this by normalizing each metric by product. We would have values between 0 and 1, where 0 would indicate no votes for that review and 1 would tell us that this is the most upvoted product review. This metric assumes, that all reviews of specific products had equal opportunity to be upvoted. This is probably related to numerous other factors, like which comments are highlighted by Steam, how many people were active when that review was written etc., but it still gives us some insight into how well a review was written for a specific product. The downside is that products with few views are treated equally as products with many views. This is problematic because it is much harder to write the best (or close to the best) review when there are more reviews. Values in less popular products would therefore be much higher than others.

So far we proposed two metrics, one that evaluates reviews overall, ignoring that some may be less viewed than others, and the second that acknowledges this fact but skews less viewed votes upwards. Is it possible to get a better metric that is a compromise of the two? To do this I propose a metric that is calculated using information about how upvoted a review was to others of the same product, and all other reviews. It should regard both values equally, so it is calculated as their sum, divided by two.

In [4]:
def clip_column(df, column_name, quantile=0.999, new_column_name=None):
    """
    When a value in a specified column falls outside the specified quantile, make it equal to the largest value in the specified quantile.
    This is used to clip big outliers for 
    """
    if new_column_name is None:
        new_column_name = column_name
    cutoff_value = df.select(column_name).quantile(0.999)
    return pl.when(pl.col(column_name) > cutoff_value).then(cutoff_value).otherwise(pl.col(column_name)).alias(new_column_name), cutoff_value

In [5]:
found_funny_expr, found_funny_cutoff_value = clip_column(df, 'found_funny', new_column_name='found_funny_cutoff')
found_awarding_expr, found_awarding_cutoff_value = clip_column(df, 'found_awarding', new_column_name='found_awarding_cutoff')
found_helpful_expr, found_helpful_cutoff_value = clip_column(df, 'found_helpful', new_column_name='found_helpful_cutoff')

In [6]:
df = df.with_columns(
    found_funny_expr,
    found_awarding_expr,
    found_helpful_expr
)

In [8]:
# create normalized metrics
df = df.with_columns(
    (
        (
            (pl.col("found_funny_cutoff") / pl.col("found_funny_cutoff").max()) + 
            (pl.col("found_funny_cutoff") / pl.col("found_funny_cutoff").max()).over("product_id")
        ) / 2).fill_nan(0.0).alias("found_funny"),
    (
        (
            (pl.col("found_helpful_cutoff") / pl.col("found_helpful_cutoff").max()) + 
            (pl.col("found_helpful_cutoff") / pl.col("found_helpful_cutoff").max()).over("product_id")
        ) / 2).fill_nan(0.0).alias("found_helpful"),
    (
        (
            (pl.col("found_awarding_cutoff") / pl.col("found_awarding_cutoff").max()) + 
            (pl.col("found_awarding_cutoff") / pl.col("found_awarding_cutoff").max()).over("product_id")
        ) / 2).fill_nan(0.0).alias("found_awarding")#,
    #(pl.col("price") / pl.col("price").max()).fill_nan(0.0)
)

In [9]:
df = df.drop(["found_funny_cutoff", "found_helpful_cutoff", "found_awarding_cutoff"])

In [10]:
df.describe()

statistic,product_id,review_text,recommended,found_awarding,found_helpful,found_funny
str,f64,str,str,f64,f64,f64
"""count""",4.4143476e7,"""44143476""","""44143476""",4.4143476e7,4.4143476e7,4.4143476e7
"""null_count""",0.0,"""0""","""0""",0.0,0.0,0.0
"""mean""",749954.496783,null,null,0.005895,0.010092,0.00686
"""std""",620549.6959,null,null,0.050572,0.055116,0.049868
"""min""",20.0,"""""","""0""",0.0,0.0,0.0
"""25%""",269210.0,null,null,0.0,0.0,0.0
"""50%""",573100.0,null,null,0.0,0.0,0.0
"""75%""",1.14962e6,null,null,0.0,0.004,0.0
"""max""",3.11778e6,"""󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰󠇰…","""1""",1.0,1.0,1.0


# Investigating review_text

In [62]:
print(list(string.ascii_lowercase) + list(string.ascii_uppercase))
i = 0
for review in df.select(['review_text']).sample(10).iter_rows():
    print(f'"{review[0]}"')
    #print(review[0].contains(list(string.ascii_lowercase) + list(string.ascii_uppercase)))
    i += 1
    if i == 10:
        break

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
"hi"
"yes it is a fantastic game"
"Well, I cant say this game is either good or bad. During tutorial, I feels that it will be fun to play, so I decide to play Online, and they say I'm require to have at least one character to join online multiplayer. So it bring me to the shop, I selected one character and click buy with the starting in game currency that they provide, the process for it to comfirm took so long until I disconnect from the server. When I login, I noticed that my in game currency have been deducted but they didn't give me the character that i selected. So, since the in game currency they give allow you to buy two character, I still maintain my positive attitude, and try purchase another character with the balance c

In [13]:
df.shape

(43163082, 6)

In [12]:
# Filter out reviews that don't contain any letters.
df = df.filter(pl.col('review_text').str.contains_any(list(string.ascii_lowercase) + list(string.ascii_uppercase)))

In [14]:
# split into train test dev
np.random.seed(15)
df_split = df.select("product_id").unique("product_id").sort("product_id")
df_split = df_split.with_columns(
    pl.lit(np.random.rand(df_split.height)).alias("split")
)
df_split = df_split.with_columns(
    pl.when(pl.col("split") < 0.8).then(pl.lit("train")).otherwise(pl.when(pl.col("split") < 0.9).then(pl.lit("test")).otherwise(pl.lit("dev"))).alias("split")
)
df_dict = df.join(df_split, on="product_id", how="left").partition_by("split", as_dict=True, include_key=False)
# df_split["split"].value_counts()

In [15]:
print(df_dict[("train",)].shape[0])
print(df_dict[("test",)].shape[0])
print(df_dict[("dev",)].shape[0])

33657650
4465061
5040371


In [16]:
def write_parquet(df_dict, filename):
    df_dict[("train",)].write_parquet(filename + '_train.parquet')
    df_dict[("test",)].write_parquet(filename + '_test.parquet')
    df_dict[("dev",)].write_parquet(filename + '_dev.parquet')

write_parquet(df_dict, "data/complete_cleaned")